In [2]:
import os
import time
import pandas as pd

In [3]:
#original_dir = os.getcwd()
#print(original_dir)

project_dir = r"D:\NBANextGamePredictor" # Choose a Path to save your files (such as .ipnyb, .py, .csv)
if not os.path.exists(project_dir):
    print(f"Creating: {project_dir}")
    os.makedirs(project_dir)

os.chdir(project_dir)
print("Working directory:", os.getcwd())

Creating: D:\NBANextGamePredictor
Working directory: D:\NBANextGamePredictor


In [4]:
subdirectory = os.path.join(project_dir, "csvs")
if not os.path.exists(subdirectory):
    print(f"Creating subdirectory for CSVs: {subdirectory}")
    os.makedirs(subdirectory)

Creating subdirectory for CSVs: D:\NBANextGamePredictor\csvs


In [5]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = ["2021-22", "2022-23", "2023-24"]
allSeasons = []

for season in seasons:
    gameFinder = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable="Regular Season")
    games = gameFinder.get_data_frames()[0]
    allSeasons.append(games)
    time.sleep(2)
allSeasonsConcat = pd.concat(allSeasons, ignore_index=True)
#print(allSeasonsConcat)

allSeasonsDir = os.path.join(subdirectory, 'regular_season.csv')
allSeasonsConcat.to_csv(allSeasonsDir, index=False)

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = ["2021-22", "2022-23", "2023-24"]
teams_abbr = [
    "BOS", "BKN", "NYK", "PHI", "TOR",
    "CHI", "CLE", "DET", "IND", "MIL",
    "ATL", "CHA", "MIA", "ORL", "WAS",
    "DEN", "MIN", "OKC", "POR", "UTA",
    "GSW", "LAC", "LAL", "PHX", "SAC",
    "DAL", "HOU", "MEM", "NOP", "SAS"
]
games = []

for season in seasons:
    gameFinder = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable="Regular Season")
    seasonGames = gameFinder.get_data_frames()[0]
    seasonGames = seasonGames[seasonGames['TEAM_ABBREVIATION'].isin(teams_abbr)]
    games.append(seasonGames)
    time.sleep(2)

games = pd.concat(games, ignore_index=True)
games.to_csv(os.path.join(subdirectory, 'all_games_raw.csv'), index=False)

In [6]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = ["2021-22", "2022-23", "2023-24"]
teams_abbr = [
    "BOS", "BKN", "NYK", "PHI", "TOR",
    "CHI", "CLE", "DET", "IND", "MIL",
    "ATL", "CHA", "MIA", "ORL", "WAS",
    "DEN", "MIN", "OKC", "POR", "UTA",
    "GSW", "LAC", "LAL", "PHX", "SAC",
    "DAL", "HOU", "MEM", "NOP", "SAS"
]

# Directory to save team files
output_dir = "team_files"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Loop through each team and save stats to separate files
for team_abbr in teams_abbr:
    team_games = games[games['TEAM_ABBREVIATION'] == team_abbr]  # Filter data for the team
    team_file_path = os.path.join(output_dir, f"{team_abbr}_games_stats.csv")  # Set file path
    team_games.to_csv(team_file_path, index=False)  # Save to CSV

In [7]:
allSeasons_OneRow = allSeasonsConcat.groupby("GAME_ID")
allSeasons1RowDF = pd.DataFrame(allSeasons_OneRow)
allSeasonsDir = os.path.join(subdirectory, 'regular_season_one_row.csv')
allSeasons1RowDF.to_csv(allSeasonsDir, index=False)

In [8]:
T1 = allSeasons_OneRow.nth(0).add_prefix("T1_")
T2 = allSeasons_OneRow.nth(1).add_prefix("T2_")

allSeasons1RowDF = pd.DataFrame(T1)
allSeasonsDir = os.path.join(subdirectory, 'regular_season_Team_1.csv')
allSeasons1RowDF.to_csv(allSeasonsDir, index=False)

allSeasons1RowDF = pd.DataFrame(T2)
allSeasonsDir = os.path.join(subdirectory, 'regular_season_Team_2.csv')
allSeasons1RowDF.to_csv(allSeasonsDir, index=False)

In [9]:
mergedData = pd.merge(T1, T2, left_on="T1_GAME_ID", right_on="T2_GAME_ID").reset_index()
mergedDir = os.path.join(subdirectory, 'merged_data.csv')
mergedData.to_csv(mergedDir, index=False)


In [10]:
columns_to_keep = [
    "T1_GAME_ID", "T1_WL", "T1_PTS", "T1_FGM", "T1_FGA", "T1_FG_PCT", "T1_FG3M", "T1_FG3A", 
    "T1_FG3_PCT", "T1_FTM", "T1_FTA", "T1_FT_PCT", "T1_OREB", "T1_DREB", "T1_AST", "T1_TOV", 
    "T1_PF", 
    "T2_WL", "T2_PTS", "T2_FGM", "T2_FGA", "T2_FG_PCT", "T2_FG3M", "T2_FG3A", "T2_FG3_PCT",
    "T2_FTM", "T2_FTA", "T2_FT_PCT", "T2_OREB", "T2_DREB", "T2_AST", "T2_TOV", "T2_PF"
]
filteredData = mergedData[columns_to_keep]
filteredDir = os.path.join(subdirectory, 'filtered_data.csv')
filteredData.to_csv(filteredDir, index=False)